In [2]:
import autogen
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model":["gpt-4o-mini"],
    },
)
gpt_api_key = config_list[0]["api_key"]

In [3]:
import os
os.environ["OPENAI_API_KEY"]=gpt_api_key

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [6]:
loaders = [PyPDFLoader('./data/whitepaper-v3.pdf')]
docs=[]
for l in loaders:
    docs.extend(l.load())
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_spliter.split_documents(docs)

In [7]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OpenAIEmbeddings()

)
vectorstore.add_documents(docs)

['9a5e8b14-ecb7-4fcf-b355-41048be988e8',
 '60b4d07f-316a-4f91-bb8b-899307bcb4d4',
 'fbee4c99-07c5-4e19-b6b0-89c68cc960ff',
 '14c4fa85-0bd3-41e9-90d3-ee99cfee495f',
 '1d8a4d8b-5842-405d-97f4-dba3d7320e76',
 'bf254d4a-72ad-4579-9df7-a57afc51690d',
 '1f10e693-1f47-4ca7-819c-72212acf822c',
 '3dc10430-148b-4a16-8cdb-ea4da7d32e4f',
 '83bfd4d0-5015-441d-9736-33da2b0335bf',
 '1b4e6faf-a169-4762-a7b7-995812be7846',
 'a8f5677f-eae8-4d4c-8e64-a2ee3ca5bf3a',
 '506b9541-f7dc-4281-a3d0-3bd3bcb5ddb8',
 'b24f4c8b-2086-4feb-a2ee-552e41b6a1cd',
 '456cd3e4-ab08-457d-b716-4b76130315ed',
 '4361e3c6-ab6a-4001-b309-961c5cc8bfeb',
 'e2a3fd0e-d156-4040-8a14-576b9e936fa1',
 '6c7b0d96-9d58-4b4a-8560-96deadf5b1a8',
 '72a88ad1-44b0-4e1f-8f8e-ea09f727d914',
 '57e1fabd-ccc9-4702-88d7-1a393d68b293',
 '47a65a9a-42cf-407d-a1bf-772518db68b3',
 'bb97cab5-7101-4cfa-a16f-f2574a1c34ac',
 '7f689bf3-0d50-4f5d-ac11-1d5838e4e22f',
 '68b42126-277f-4988-97b2-94fcae201e61',
 '68724851-c1e2-4ba4-9161-ab4262e3c424',
 '7fbde98c-d784-

In [14]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    retriever=vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history",return_messages=True)
)

In [15]:
result = qa(({"question":"What is uniswap?"}))
result

{'question': 'What is uniswap?',
 'chat_history': [HumanMessage(content='What is uniswap?'),
  AIMessage(content=' Uniswap is a noncustodial automated market maker implemented for the Ethereum Virtual Machine. It provides increased capital efficiency and fine-tuned control to liquidity providers, improves the accuracy and convenience of the price oracle, and has a more flexible fee structure. It was created by Hayden Adams, Noah Zinsmeister, Moody Salem, River Keefer, and Dan Robinson in March 2021.')],
 'answer': ' Uniswap is a noncustodial automated market maker implemented for the Ethereum Virtual Machine. It provides increased capital efficiency and fine-tuned control to liquidity providers, improves the accuracy and convenience of the price oracle, and has a more flexible fee structure. It was created by Hayden Adams, Noah Zinsmeister, Moody Salem, River Keefer, and Dan Robinson in March 2021.'}

In [16]:
def answer_uniswap_question(question):
    response = qa({"question": question})
    return response["answer"]

In [23]:
# 用function call设置AutoGen的user agent和assistant agent
llm_config={
    "request_timeout":600,
    "seed":42,
    "config_list":config_list,
    "temperature":0,
    "functions":[
        {
            "name":"answer_uniswap_question",
            "description":"Answer any Uniswap ralated questions",
            "parameters": {
                "type":"object",
                "properties":{
                    "question":{
                        "type":"string",
                        "description":"The question to ask in relation to Uniswap protocol",
                    }
                },
                "required":["question"],
            },
        }
    ],
}

In [25]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

uesr_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,#避免无限对话循环
    code_execution_config={"work_dir":"."},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction. Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"answer_uniswap_question":answer_uniswap_question}
)

In [26]:
uesr_proxy.initiate_chat(
    assistant,
    message="""
I'm writing a blog to introduce the version 3 of Uniswap protocol, Find the answers to the 3 questions below and write a introduction based on the answers.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Please start the work now.
"""
)

user_proxy (to assistant):


I'm writing a blog to introduce the version 3 of Uniswap protocol, Find the answers to the 3 questions below and write a introduction based on the answers.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Please start the work now.


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: answer_uniswap_question *****
Arguments: 
{"question":"What is Uniswap?"}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION answer_uniswap_question...
user_proxy (to assistant):

***** Response from calling function "answer_uniswap_question" *****
 Uniswap v3 is a noncustodial automated market maker implemented for the Ethereum Virtual Machine. It provides increased capital efficiency and fine-tuned control to liquidity prov